In [ ]:
import sys
sys.path.append('../')

import cigarmath as cm

# Clipping

## Basics

When performing a local-alignment you are asking which **regions** of the query map to which **regions** of the reference.
This often leads to instances where the leading and trailing ends of the query sequence are not included in the alignment.
Take the example below:

In [ ]:
cigartuples = cm.cigarstr2tup('5S9M4S')
cigartuples

[(4, 5), (0, 9), (4, 4)]

`cigarmath` provides a number of functions to handle clippings.

`left_clipping` and `right_clipping` returns the upstream and downstream clipping respectively.

In [ ]:
cm.left_clipping(cigartuples)

5

In [ ]:
cm.right_clipping(cigartuples)

4

`declip` will remove left and right clippings returning only the _meat_ of the alignment.

In [ ]:
cm.declip(cigartuples)

[(0, 9)]

You can also provide a set of sequences or lists along with your cigartuples and they will be clipped accordingly.
This is useful if you are trying to isolate the sequence relevant to the alignment.

In [ ]:

# Hypothetical sequence and quality scores
seq = 'xxxxxHIJKLMNOPyyyy'
quals = [1,1,1,1,1,3,3,3,3,3,3,3,3,3,2,2,2,2]

# Provide along with the cigartuples
cm.declip(cigartuples, seq, quals)

5 -4


([(0, 9)], 'HIJKLMNOP', [3, 3, 3, 3, 3, 3, 3, 3, 3])

## Hard Clipping

The `xxxxx` and `yyyy` ends are not considered part of the alignment.
In Next Generation Sequencing (NGS) this is usually due to adapters added to either side to facilitate sequencing.
In long-read sequencing it can also be present when a query's alignment is split arross multiple alignments.
Take the example below:

This is called a _supplementary alignment_.
In this instance instead of providing one large alignment, the algorithm has instead produced two local alignments.
In the case of most tools, it will also **hard clip** the sequence by removing the unneeded sequence from the query sequence stored in the SAM/BAM file.
This can save a great deal of space.

`is_hard_clipped` can be used to determine whether a sequence has been hard_clipped.

In [ ]:
cm.is_hard_clipped(cm.cigarstr2tup('5H9M4H'))

True

## Clippify

When performing a **global** alignment with long-read data, sometimes due to the addition of adapters,
the ends of the alignment are untrust-worthy.
This can be corrected by approximating a local-alignemnt from this global alignment.

In the above the lowercase query letters are _technical sequences_ and not relevant to the alignment.
However, global alignment tools like `muscle` and `mafft` cannot clip these ends.
`cm.softclipify` wsa designed for this purpose.

This tool will read _in_ from each end, and consume cigartuples until it reaches a **mapping** block of a definable size.
Then, all end sequences are marked as soft-clipped.
Reconsider our example:

In [ ]:
new_tuples, new_offset = cm.softclipify(cm.cigarstr2tup('3I4D10M4D2M3I'))
print('Reference Offset:', new_offset)
print('CIGAR:', cm.cigartup2str(new_tuples))

Reference Offset: 4
CIGAR: 3S10M4D2M3S


In [ ]:
# You can also specify the size of the mapping block that will 'lock' the local alignment
new_tuples, new_offset = cm.softclipify(cm.cigarstr2tup('3I4D10M4D2M3I'),
                                        required_mapping=4)
print('Reference Offset:', new_offset)
print('CIGAR:', cm.cigartup2str(new_tuples))

Reference Offset: 4
CIGAR: 3S10M5S


# Conclusion

Together, these functions provide a number of useful building blocks for more complex algorithms when handling clipping in CIGARs.